# Revocation - Alice

## Role: Issuer, Verifier & Revoker

There will be two parts to this notebook:

- Part 1: Alice issues a revocable credential to [Bob](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb). Bob will store this credential and present proof of it back to Alice, where she will verify the presentation.
- Part 2: Alice revokes the credential issued to Bob in Part 1 and requests another proof from Bob. This second presentation from Bob will fail to verify.

For details on how revocation works on Hyperledger Indy, you can read more [here](https://github.com/hyperledger/indy-hipe/tree/master/text/0011-cred-revocation).

# Part 1 - Issuing & Proving Non-Revoked Credential

## 1. Initiate the controller for Alice

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)

## 2. Register Listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

In [ ]:
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}

agent_controller.register_listeners([cred_listener, proof_listener], defaults=True)

## 3. Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script `create_connection.py` in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://127.0.0.1:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb)
* [Bob](http://127.0.0.1:8889/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb)

In [ ]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")

## 4. Write a Schema to the Ledger

**Note: You will only be able to do this once unless you change the `schema_name` or `schema_version`. Or tear down the current ledger using `./manage down`**

In [ ]:
# Define you schema name - must be unique on the ledger
schema_name = "test_revocable_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill", "age"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

## 5. Write a Revocable Credential Definition to the Ledger

By adding the `support_revocation` flag, the ACA-PY agent under the hood knows to create some additional structures to support revocation of all credentials issued under this credential definition.

Specifically the agent creates what is known as a revocation registry, and writes this to the public ledger. You should be able to see these transactions in the ledger explorer at [localhost:9000](http://localhost:9000/browse/domain).

It also adds a tails file to the tail server that needs to be accessible by those creating and verifying proofs of non-revocation.

In [ ]:
response = await agent_controller.definitions.write_cred_def(schema_id, support_revocation=True)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

## 6. Populate the Attribues to Issue to Bob

The schema defines two attributes: name and skill. Both must be populated with attribute values that Alice wishes to issue in a credential to Bob. To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [ ]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "skill", "value": "researcher"},
    {"name": "age", "value": "21"}
]
print(credential_attributes)

## 7. Continue in [Bob's Notebook](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb)

You need to initialise the controller and listen to webhooks so you can track the messages Bob's agent receives.

## 9. Issue Revocable Credential

Note: This step is exactly the same as issuing an unrevocable credential. Revocable credentials are those issued under cred_defs (public keys) that have been appropriately set up to support the revocation cryptography.

In [ ]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

## 10. Continue in [Bob's Notebook ](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb)

Here you will request, store and then present the credential.

## 14. Send Proof Request

Request Bob to prove that he is a researcher.

Note: This step is very similar to the [Present Proof tutorial](http://127.0.0.1:8888/notebooks/Part%206%20-%20Present%20Proof.ipynb). You will notice some extra fields such as `non_revoked`, which corresponds to the proof requester specifying a requirement for the credential to not be revoked up until a certain timestamp.

In [ ]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof of Skill from Bob")
#Set some variables

SELF_ATTESTED = True
exchange_tracing = False

# Either the attribute can be specified to be non revoked
req_attrs = [
    {
        "name": "skill",
        "restrictions": [{"issuer_did": issuer_did}],
        "non_revoked": {"to": int(time.time() - 1)},
    }
]

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {},
}


## Or the overall request itself
indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

response = await agent_controller.proofs.send_request(proof_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

## 15. Continue in [Bob's Notebook ](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb)

Bob will respond to the presentation request

## 17. Verify the Presentation

For a non-revoked credential, `verify['verified']` should be `'true'`.

In [ ]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
verify

In [ ]:
"""
Note that verify['state'] is ALWAYS going to be 'verified'.
verify['verified'] is the actual indicator of whether this proof
has succeeded, or if it has already been revoked and the 
proof has failed.
"""
print("Verified: {}".format(verify['verified'] == 'true'))
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])

# Part 2 - Revoking the Credential

## 18. Revoke the Credential

Alice revokes the credential by providing the `cred_ex_id`.

In [ ]:
# For revoke_credential() you only need to provide (cred_ex_id) OR (rev_reg_id AND cred_rev_id).
response = await agent_controller.revocations.revoke_credential(record_id, publish=True)
response

## 19. Continue in [Bob's Notebook ](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb)

Bob will check his credential to find that it is now revoked.

## 20. Send request for proof of skill

Alice will request Bob to prove his skill once again, similar to Part 1.

In [ ]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof of Skill from Bob")
#Set some variables

SELF_ATTESTED = True
exchange_tracing = False

# Either the attribute can be specified to be non revoked
req_attrs = [
    {
        "name": "skill",
        "restrictions": [{"issuer_did": issuer_did}],
        "non_revoked": {"to": int(time.time() - 1)},
    }
]

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {},
}


## Or the overall request itself
indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

response = await agent_controller.proofs.send_request(proof_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

## 21. Continue in [Bob's Notebook ](http://127.0.0.1:8889/notebooks/Part%2010%20-%20Revocation.ipynb)

Bob will respond to the presentation request

## 23. Verify the Presentation

For a revoked credential, `verify['verified']` should be `'false'`.

In [ ]:
# verify the presentation
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
verify

In [ ]:
"""
Note that verify['state'] is ALWAYS going to be 'verified'.
verify['verified'] is the actual indicator of whether this proof
has succeeded, or if it has already been revoked and the 
proof has failed.
"""
print("Verified: {}".format(verify['verified'] == 'true'))
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])

## End of Tutorial

Be sure to terminate the controller.

In [ ]:
response = await agent_controller.terminate()
print(response)